---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
#print(pd.read_csv('readonly/train.csv', encoding='cp1252').columns)
#print()
#print(pd.read_csv('readonly/test.csv').isnull().sum())
#print(pd.read_csv('readonly/test.csv').mean())

In [2]:
#df.columns
#df.isnull().sum()

In [3]:
#df[['agency_name', 'compliance']].groupby('agency_name').agg(['mean', 'sum'])

In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

def get_Xy(file = 'train.csv', readonly=False):
    
    def get_df(file = 'train.csv', readonly = False):
        df_initial = pd.read_csv(file, encoding='cp1252', dtype={'violation_street_number': int, 
                                                                                 'non_us_str_code':str, 'zip_code': str, 
                                                                                 'grafitti_status':str})
        df_initial.drop(['violation_zip_code', 'non_us_str_code', 'grafitti_status'], 
                        axis=1, inplace=True)

        if 'payment_date' in df_initial.columns:
            df_initial.drop(['payment_date', 'compliance_detail'], axis=1, inplace=True)

        #add lat and lon columns
        if readonly == True:
            addresses = pd.read_csv('readonly/addresses.csv')
            latlons = pd.read_csv('readonly/latlons.csv')
        else:
            addresses = pd.read_csv('addresses.csv')
            latlons = pd.read_csv('latlons.csv')
        adlatlon = pd.merge(addresses, latlons, how='inner', left_on='address', right_on='address')
        df = pd.merge(df_initial, adlatlon, how='left', left_on='ticket_id', right_on='ticket_id')
        #df['Mailing_Address'] = df.apply(lambda x: '{} {}'.format(int(x['violation_street_number']), x['violation_street_name']).lower(), axis=1)
        #df['spelling'] = df.apply(lambda x: x['Mailing_Address'] in x['address'], axis=1)

        df.dropna(subset=['fine_amount'], inplace=True)

        df['agency_name'].replace({'Buildings, Safety Engineering & Env Department': 1, 
                               'Department of Public Works': 2,
                               'Detroit Police Department': 3,
                               'Health Department': 4,
                               'Neighborhood City Halls': 5},
                             inplace=True)

        return df
    
    df = get_df(file = file, readonly=readonly)
    y = 0
    
    if 'compliance' in df.columns:
        y = df[df['compliance'].isnull() == False]['compliance']
        X = df[df['compliance'].isnull() == False]
    else:
        X = df
    
    X.drop(['violation_description', 'mailing_address_str_number', 'mailing_address_str_name', 
            'violation_street_number', 'violation_street_name', 'address', 'city', 'state', 'country', 'ticket_id',
           'inspector_name', 'violator_name', 'admin_fee', 'state_fee', 'clean_up_cost', 'disposition'], 
           axis=1, inplace=True)
    
    if 'payment_amount' in X.columns:
        X.drop(['payment_amount', 'balance_due', 'payment_status', 'collection_status','compliance'], axis=1,inplace=True)
    
    X.loc[:,'violation_code_2'] = X['violation_code'].str.split('-').str[1].str.split('/').str[0]
    X.loc[:,'violation_code'] = X['violation_code'].str.split('-').str[0]
    
    X.loc[:,'zip_code'] = X['zip_code'].str.split('-').str[0].astype(str)


    #no_zip = X[X['zip_code'].str.isnumeric() == False].index
    #X.drop(no_zip, inplace=True)
    #if type(y) != int:
    #    y.drop(no_zip, inplace=True)
    
    X.loc[:,'zip_code'] = X['zip_code'].apply(lambda x: 00000 if (x.isnumeric() == False) else x)

    
    X = X.astype({'zip_code':int, 'violation_code':int, 'violation_code_2':float})
    #X.astype({'ticket_issued_date':'datetime64', 'hearing_date':'datetime64'})

    X.loc[:,'ticket_issued_date'] = pd.to_datetime(X['ticket_issued_date']) 
    X.loc[:,'hearing_date'] = pd.to_datetime(X['hearing_date'])
    
    #X['issue_day'] = pd.DatetimeIndex(X['ticket_issued_date']).day
    X['issue_month'] = pd.DatetimeIndex(X['ticket_issued_date']).month
    #X['issue_hour'] = pd.DatetimeIndex(X['ticket_issued_date']).hour
    #X['ticket_hearing_timedelta'] = X['hearing_date'] - X['ticket_issued_date']
    #X['ticket_hearing_timedelta'] = X['ticket_hearing_timedelta'].dt.days
    
    X.drop(X[X['hearing_date'] > '2011-10-01'].index)
    
    
    X.drop(['ticket_issued_date','hearing_date'],axis=1, inplace=True)
    
    X = X.fillna(method='ffill')
    if type(y) != int:
        y = y.fillna(method='ffill')
        
    X = X[['violation_code', 'zip_code', 'judgment_amount','lat', 'lon']]
    
    ##issues with zip code or fillna?
    ##is Data leaky?
    
    return X, y

#X,y = get_Xy(readonly=True)
#X


In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#X['lon'] = X['lon']*-1
#print(X[X<0])

#apply SelectKBest class to extract top 10 best features
#bestfeatures = SelectKBest(score_func=chi2, k=10)
#fit = bestfeatures.fit(X,y)
#dfscores = pd.DataFrame(fit.scores_)
#dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [6]:
#df[['ticket_issued_date', 'hearing_date','compliance']].dropna().sort_values('hearing_date')
#df[df['hearing_date'] > '2011-09-01'].loc[:, ['ticket_issued_date', 'hearing_date','compliance']]

In [7]:
#import matplotlib.pyplot as plt
#plt.figure(figsize=(12,8))
#sns.heatmap(df.corr(), annot=True)
#plt.yticks(rotation='horizontal')
#plt.xticks(rotation= 90)

In [8]:
def model_tests(readonly=False): 

    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import roc_auc_score


    if readonly == True:
        X, y = get_Xy('readonly/train.csv', readonly=readonly)
        X_final_test, y_null = get_Xy('readonly/test.csv', readonly=readonly)
    if readonly == False:
        X, y = get_Xy('train.csv', readonly=readonly)
        X_final_test, y_null = get_Xy('test.csv', readonly=readonly)

    from sklearn.model_selection import train_test_split    
    X_train, X_test, y_train, y_test = train_test_split(X,y)

    #for set in [X_train, X_test, y_train, y_test]:
    #    print(set.shape)

    #from sklearn.tree import DecisionTreeClassifier
    #dtc = DecisionTreeClassifier()
    #print('\n\tDecision Tree\nCross Val scores: {}'.format(cross_val_score(dtc, X, y, cv=4, scoring='roc_auc')))    

    #from sklearn.neighbors import KNeighborsClassifier
    #knn = KNeighborsClassifier()
    #print('\n\tKNN\nCross Val scores: {}'.format(cross_val_score(knn, X, y, cv=4, scoring='roc_auc')))    

    #from sklearn.linear_model import Lasso
    #lasso = Lasso()
    #print('\n\tLasso\nCross Val scores: {}'.format(cross_val_score(lasso, X, y, cv=4, scoring='roc_auc')))  

    #from sklearn.svm import LinearSVC
    #linearsvc = LinearSVC()
    #print('\n\tLinearSVC\nCross Val scores: {}'.format(cross_val_score(linearsvc, X, y, cv=4, scoring='roc_auc')))

    #from sklearn.svc import SVC
    #svc = SVC()
    #print('\n\tSVC\nCross Val scores: {}'.format(cross_val_score(svc, X, y, cv=4, scoring='roc_auc')))

    #from sklearn.ensemble import RandomForestClassifier
    #rf = RandomForestClassifier()
    #print('\n\tRandom Forest\nCross Val scores: {}'.format(cross_val_score(rf, X, y, cv=4, scoring='roc_auc')))  


    from sklearn.ensemble import GradientBoostingRegressor
    gbc = GradientBoostingRegressor()
    #print('\n\tGradient Boosting Classifier\nCross Val scores: {}'.format(cross_val_score(gbc, X, y, cv=4, scoring='roc_auc')))  
    gbc.fit(X_train, y_train)
    y_preds = gbc.predict(X_test)
    #print('Performance on test set: ', roc_auc_score(y_preds, y_test))

    return gbc

#model_tests(readonly=True)

In [9]:
#parameter selection
def model_gridsearch():
    readonly=True

    from sklearn.model_selection import GridSearchCV
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.metrics import roc_auc_score
    print('Imported')

    model = RandomForestClassifier(min_samples_split=170)
    print('Model variable created')

    if readonly == True:
        X, y = get_Xy('readonly/train.csv', readonly=readonly)
    if readonly == False:
        X, y = get_Xy('train.csv', readonly=readonly)
    print('X,y values loaded')

    from sklearn.model_selection import train_test_split    
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    print('Train Test split')

    grid_values = {'max_features': [0.3, 0.5, 0.7], 'n_estimators': [3, 10, 20]}
    grid_model_auc = GridSearchCV(model, param_grid = grid_values, scoring = 'roc_auc', cv=3, n_jobs=-1)
    print('GridSeachCV variable created')
    grid_model_auc.fit(X, y)
    print('GridSeachCV fitted')
        
    print('Grid best parameter (max. AUC): ', grid_model_auc.best_params_)
    print('Grid best score (AUC): ', grid_model_auc.best_score_)

    test_score = grid_model_auc.predict(X_test)
    print('Grid predict test score (AUC): ', roc_auc_score(y_test, test_score))    
    
    return grid_model_auc
 
#grid = model_gridsearch()
#results = pd.DataFrame(grid.cv_results_)
#results

In [10]:
#import seaborn as sns
#%matplotlib notebook
#sns.heatmap(results.pivot_table(columns='param_max_features', 
#                                index='param_n_estimators', 
#                                values='mean_test_score'), annot=True)

In [11]:
#import seaborn as sns
#%matplotlib notebook
#sns.heatmap(results.pivot_table(columns='param_max_features', index='param_n_estimators', values='mean_test_score'), annot=True)

In [12]:
def get_model(readonly = False):
    from sklearn.ensemble import RandomForestClassifier
    
    X,y = get_Xy(readonly=readonly)
    
    model = RandomForestClassifier(max_features=0.3, n_estimators=20, min_samples_split=170).fit(X,y)
    
    return model

In [13]:
#feature engineering

In [15]:
def blight_model(readonly=False):
    
    if readonly == True:
        X_final_test, y_null = get_Xy('readonly/test.csv', readonly=readonly)
    if readonly == False:
        X_final_test, y_null = get_Xy('test.csv', readonly=readonly)
    
    preds=model_tests(readonly=readonly).predict(X_final_test)
    
    return pd.Series(preds, index = pd.read_csv('test.csv')['ticket_id'])

blight_model(readonly=True)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.

FileNotFoundError: File b'test.csv' does not exist